In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline


import os

data_path = ''

/home/fterroso/python/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
import warnings
warnings.filterwarnings("ignore")

## Read clusters' members

In [3]:
cluster_members_twt_df = pd.read_csv(os.path.join('data', 'Cluster Provincias Twitter.csv'), sep=';', index_col=0)

In [4]:
cluster_members_twt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 1 to 52
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   codprov               51 non-null     object
 1   latitud               52 non-null     object
 2   longitud              52 non-null     object
 3   altitud               52 non-null     object
 4   poblacion             52 non-null     int64 
 5   poblacionporcentaje   52 non-null     object
 6   densidadpoblacionkm   52 non-null     object
 7   superfice             52 non-null     object
 8   superficieporcentaje  52 non-null     object
 9   degree                52 non-null     int64 
 10  Cluster               52 non-null     int64 
dtypes: int64(3), object(8)
memory usage: 4.9+ KB


In [5]:
cluster_members_twt_df.head()

,codprov,latitud,longitud,altitud,poblacion,poblacionporcentaje,densidadpoblacionkm,superfice,superficieporcentaje,degree,Cluster
Id,,,,,,,,,,,
1,VI,"42,84641","-2,667893","534,3582",331549,"0,71","109,17",3037,"0,6",69,0
2,AB,"38,99765","-1,86007","685,9573",388167,"0,83","26,01",14926,"2,95",51,1
3,A,"38,3452","-0,481006","16,63256",1858683,"3,95","319,53",5817,"1,15",71,1
4,AL,"36,84016","-2,467922","27,00703",716820,"1,52","81,69",8775,"1,73",62,1
5,AV,"40,65642","-4,700323","1129,841",157640,"0,34","19,58",8050,"1,59",52,0


In [6]:
cluster_members_twt_df['Cluster'].nunique()

7

### Cluster in INE

In [7]:
cluster_members_ine_df = pd.read_csv(os.path.join('data', 'Cluster Provincias Ministerio.csv'), sep=';', index_col=0)

In [8]:
cluster_members_ine_df.head()

,codprov,latitud,longitud,altitud,poblacion,poblacionporcentaje,densidadpoblacionkm,superfice,superficieporcentaje,degree,Cluster
Id,,,,,,,,,,,
1,VI,42.84641,-2.667893,534.3582,331549,0.71,109.17,3037.0,0.60,13,0
26,LO,42.46577,-2.449995,396.4728,316798,0.67,62.79,5045.0,1.00,9,0
9,BU,42.34087,-3.699731,865.8016,356958,0.76,25.46,14022.0,2.77,11,0
10,CC,39.47618,-6.370760,434.6162,394151,0.84,19.84,19868.0,3.93,10,1
6,BA,38.87860,-6.970284,192.0375,673559,1.43,30.95,21766.0,4.30,11,1


In [9]:
cluster_members_ine_df['Cluster'].nunique()

6

In [10]:
from datetime import date, timedelta

def generate_target_days_fn(init_date, final_date):

    delta = e_date - i_date       # as timedelta

    target_days = []
    for i in range(delta.days + 1):
        day = i_date + timedelta(days=i)
        target_days.append(day)
    
    return target_days

In [11]:
import datetime
e_date = datetime.datetime.strptime('2020-06-20', '%Y-%m-%d')
i_date = datetime.datetime.strptime('2020-04-01', '%Y-%m-%d')
target_days = generate_target_days_fn(i_date, e_date)

## Generate cluster dataframes

In [12]:
import itertools
def generate_cluster_od(_od, cluster_members_df, column_type=str):
    cluster_ids = cluster_members_df['Cluster'].unique()
    cluster_od_df = pd.DataFrame(0, index= cluster_ids, columns=cluster_ids)


    for c_o, c_d in itertools.combinations(cluster_ids, r=2):
        _members_co= [column_type(m) for m in cluster_members_df[cluster_members_df['Cluster']==c_o].index.tolist()]
        _members_cd= [str(m) for m in cluster_members_df[cluster_members_df['Cluster']==c_d].index.tolist()]
        
        #print(_members_co, _members_cd, _od.index)
        
        od_clusters=_od.loc[_members_co][_members_cd]
        od_clusters['total'] = od_clusters.sum(axis=1)
        od_clusters.loc['total'] = od_clusters.sum()
        
        cluster_od_df.loc[c_o,c_d]= od_clusters.loc['total','total']
        
        _members_co= [str(m) for m in cluster_members_df[cluster_members_df['Cluster']==c_o].index.tolist()]
        _members_cd= [column_type(m) for m in cluster_members_df[cluster_members_df['Cluster']==c_d].index.tolist()]
        
        od_clusters=_od.loc[_members_cd][_members_co]
        od_clusters['total'] = od_clusters.sum(axis=1)
        od_clusters.loc['total'] = od_clusters.sum()
        
        cluster_od_df.loc[c_d,c_o]= od_clusters.loc['total','total']
        
    cluster_od_df['total'] = cluster_od_df.sum(axis=1)
    cluster_od_df.loc['total'] = cluster_od_df.sum()
    return cluster_od_df

In [13]:
def map_ODs_to_clusters_fn(source_type, cluster_members_df):
    for date in tqdm_notebook(target_days):
        od_date= pd.read_csv(os.path.join('data', 'OD', 'date_{}_OD_{}_prov.csv'.format(source_type,date.strftime('%Y-%m-%d'))), 
                            index_col=0)
        cluster_od= generate_cluster_od(od_date, cluster_members_df)
        cluster_od.to_csv(os.path.join('data', 'OD', 'date_{}_OD_{}_prov_cluster.csv'.format(source_type,date.strftime('%Y-%m-%d'))))
    
    od_global= pd.read_csv(os.path.join('data', 'OD', 'global_{}_OD_prov.csv'.format(source_type)))
    g_cluster_od= generate_cluster_od(od_global, cluster_members_df, column_type=int)
    g_cluster_od.to_csv(os.path.join('data', 'OD', 'global_{}_OD_prov_cluster.csv'.format(source_type)))                                    

In [14]:
map_ODs_to_clusters_fn('TWT', cluster_members_twt_df)

In [15]:
map_ODs_to_clusters_fn('INE', cluster_members_ine_df)

# Map COVID-19 cases to CPROVs

In [16]:
covid_cases_df = pd.read_csv(os.path.join(data_path,'Spain-INE-covid-mob-study-mar-jun-20','COVID datos_provincias.csv'), sep=';')

In [17]:
target_covid_cases_df = covid_cases_df[covid_cases_df['mes'].isin([4, 5, 6])]

In [18]:
target_covid_cases_df.head()

,Provincia_cod,dia,mes,num_casos
61,1,1,4,52
62,1,2,4,46
63,1,3,4,70
64,1,4,4,25
65,1,5,4,33


In [19]:
def map_covid_cases_to_clusters_fn(covid_cases_df, clusters_df):
    cases_per_cluster=[]
    for c in clusters_df['Cluster'].unique():
        cluster_members_df = clusters_df[clusters_df['Cluster']==c]
        member_ids = cluster_members_df.index
        cluster_covid_cases_df = covid_cases_df[covid_cases_df['Provincia_cod'].isin(member_ids)]
        for month in [4, 5, 6]:
            days_list = cluster_covid_cases_df[cluster_covid_cases_df['mes']==month]['dia'].unique().tolist()
            days_list.sort()
            for day in days_list:
                n_cases = cluster_covid_cases_df[(cluster_covid_cases_df['mes']==month) & 
                                                 (cluster_covid_cases_df['dia']==day)]['num_casos'].sum()
                cases_per_cluster.append((c,day,month,n_cases))
    cases_per_cluster_df = pd.DataFrame.from_records(cases_per_cluster, columns=covid_cases_df.columns)
    return cases_per_cluster_df

### Map to INE clusters

In [20]:
cases_per_ine_cluster_df = map_covid_cases_to_clusters_fn(target_covid_cases_df,cluster_members_ine_df)

In [21]:
cases_per_ine_cluster_df.head()

,Provincia_cod,dia,mes,num_casos
0,0,1,4,2930
1,0,2,4,2366
2,0,3,4,2341
3,0,4,4,1941
4,0,5,4,1643


In [22]:
"""
cases_per_cluster=[]
for c in cluster_members_ine_df['Cluster'].unique():
    cluster_members_df = cluster_members_ine_df[cluster_members_ine_df['Cluster']==c]
    member_ids = cluster_members_df.index
    cluster_covid_cases_df = target_covid_cases_df[target_covid_cases_df['Provincia_cod'].isin(member_ids)]
    for month in [4, 5, 6]:
        days_list = cluster_covid_cases_df[cluster_covid_cases_df['mes']==month]['dia'].unique().tolist()
        days_list.sort()
        for day in days_list:
            n_cases = cluster_covid_cases_df[(cluster_covid_cases_df['mes']==month) & 
                                             (cluster_covid_cases_df['dia']==day)]['num_casos'].sum()
            cases_per_cluster.append((c,day,month,n_cases))

cases_per_cluster_df = pd.DataFrame.from_records(cases_per_cluster, columns=target_covid_cases_df.columns)
"""


"\ncases_per_cluster=[]\nfor c in cluster_members_ine_df['Cluster'].unique():\n    cluster_members_df = cluster_members_ine_df[cluster_members_ine_df['Cluster']==c]\n    member_ids = cluster_members_df.index\n    cluster_covid_cases_df = target_covid_cases_df[target_covid_cases_df['Provincia_cod'].isin(member_ids)]\n    for month in [4, 5, 6]:\n        days_list = cluster_covid_cases_df[cluster_covid_cases_df['mes']==month]['dia'].unique().tolist()\n        days_list.sort()\n        for day in days_list:\n            n_cases = cluster_covid_cases_df[(cluster_covid_cases_df['mes']==month) & \n                                             (cluster_covid_cases_df['dia']==day)]['num_casos'].sum()\n            cases_per_cluster.append((c,day,month,n_cases))\n\ncases_per_cluster_df = pd.DataFrame.from_records(cases_per_cluster, columns=target_covid_cases_df.columns)\n"

In [24]:
cases_per_ine_cluster_df.tail()

,Provincia_cod,dia,mes,num_casos
541,5,26,6,1
542,5,27,6,0
543,5,28,6,0
544,5,29,6,0
545,5,30,6,5


In [25]:
cases_per_ine_cluster_df.to_csv(os.path.join('data', 'COVID-19_cases_per_ine_cprov.csv'))

### Map to TWT clusters

In [26]:
cases_per_twt_cluster_df = map_covid_cases_to_clusters_fn(target_covid_cases_df,cluster_members_twt_df)

In [27]:
cases_per_twt_cluster_df.head()

,Provincia_cod,dia,mes,num_casos
0,0,1,4,2930
1,0,2,4,2366
2,0,3,4,2341
3,0,4,4,1941
4,0,5,4,1643


In [28]:
cases_per_twt_cluster_df.tail()

,Provincia_cod,dia,mes,num_casos
632,5,26,6,0
633,5,27,6,0
634,5,28,6,0
635,5,29,6,0
636,5,30,6,0


In [29]:
cases_per_twt_cluster_df.to_csv(os.path.join('data', 'COVID-19_cases_per_twt_cprov.csv'))

In [30]:
print("That's all folks")

That's all folks
